# ANALYSE DES DONNEES COVID EN FRANCE ET PREDICTION DU TAUX D'OCCUPATION DES LITS
#### Dans ce notebook, nous analysons la rélation entre les variables liées à la vaccination (1ère et 2ème dose), les variables relative à la pandémie avec le taux d'occupation des lits dans les hopitaux en été en France.

### Chargement des library nécéssaires pour l'analyse des données

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Chargement de la dataset depuis Kaggle

In [ ]:
data = pd.read_csv("../input/vaccination-impact-on-covid19-indicators/Vaccine consequences on reproductibility.csv", index_col="extract_date", parse_dates=True, dayfirst=True)
df = data.copy()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().value_counts()

#### On constate qu'il y a 3 valeurs manquantes, ce que nous pouvons vérifier graphiquement avec le heatmap

In [ ]:
sns.heatmap(df.isna())

#### Comme le nombre de valeurs manquantes n'est pas très élevé, on peut simplement les supprimer.

In [ ]:
df = df.dropna(axis=0)

In [ ]:
sns.heatmap(df.isna())

#### Une fois la dataset nettoyée nous pouvons procéder à l'analyse

## ANALYSE DES DONNEES

In [ ]:
sns.pairplot(df)

#### Ce pairplot nous donne un aperçu des relations entre les différentes variables, mais une analyse plus poussée est nécéssaire. Examinons la corrélation entre ces différentes variables.

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(df.corr(), annot=True)

#### On constate que les variables liées aux doses de vaccin sont fortement corrélées entre elles et le leurs valeurs cummulées. Dans la suite de l'analyse nous allons préférer mettre en avant les valeurs cummulées qui nous donnent une tendance claire de l'évolution de la vaccination dans ses deux composantes. Notre target "Taux d'occupation" est correlé négativement avec l'augmentation des doses de vaccination. 

#### Avant d'approfondir cette analyse de corrélation, observons comment ces indicateurs sont distribués et ont évolués sur la période.

In [ ]:
var_float = df.select_dtypes("float64").columns
var_int = df.select_dtypes("int64").columns

In [ ]:
sns.set_theme()
for col in var_float:
    plt.figure()
    sns.displot(df[col], kind="hist", kde =True)
    plt.draw()

In [ ]:
sns.set_theme()
for col in var_int:
    plt.figure()
    sns.displot(df[col], kind="hist", kde =True)
    plt.draw()

#### On constate qu'aucune de ces variables ne suit une distribution normale, ce qui va nous obliger de procéder à leur normalisation lors du pre-processing.

#### Voyons à présent commens ces variables ont évoluées sur la période.

In [ ]:
for col in var_float:
    plt.figure()
    df["2020":"2021"][col].plot()
    plt.ylabel(col)
    plt.draw()

In [ ]:
for col in var_int:
    plt.figure()
    df["2020":"2021"][col].plot()
    plt.ylabel(col)
    plt.draw()

### L'anlyse de ce différents graphiques montre que le taux d'occupation des lits a baissé avec l'augmentation du taux de vaccination. Cette tendance a été cassée au milieu de l'été (à partir de juillet) en dépit d'une augmentation du taux d'incidence, du nombre de cas positifs détéctés ainsi que du taux de reproductivité du virus. La poursuite de la dynamique vaccinale tant pour le première dose que la seconde a fait qu'on observe très rapidement un plateau sur la courbe des cas positif entre mi-jillet et mi-aôut avant une tendance bassière à la fin du mois d'août. Ce constat se confirme dans les graphiques qui suivent, montrant la dépendance linéaire entre ces différents indicateurs.

In [ ]:
sns.lmplot(data = df, x = "tx_pos", y = "taux_occupation_sae")
sns.lmplot(data = df, x = "tx_incid", y = "taux_occupation_sae")

### On constate bien que plus le taux d'incidnce augmente, donc le taux de positivité, plus les lits sont occupé, cette forte pente signifie que ces nouvelles contaminations sont souvent des formes graves de la maladie, du fait que la majorité des malades sont non vaccinés et présentent les formes grave liées aux nouveaux variants.

In [ ]:
sns.lmplot(data = df, x = "n_cum_dose1", y = "taux_occupation_sae")
sns.lmplot(data = df, x = "n_cum_complet", y = "taux_occupation_sae")

### Plus le nombre de vaccinés à la première dose et à la deuxième dose augmente, plus le taux d'occupation de lits à l'hopital baisse. Cette baisse est plus importante lors le nombre de personnes ayant une vaccination complète augmente. Ceci, en depit de l'augmentation du taux de reproductivité du virus, qui montre que les contaminations augmentent, mais n'engendrent pas des formes graves du fait du grand nombre de vaccinés.

In [ ]:
sns.lmplot(data = df, x = "R", y = "taux_occupation_sae")

### On constate qu'avec l'aumentation de la vaccination, le taux d'occupation des lit baisse en dépit de l'augmentation du taux de reproductivité du virus, c'est dire le nombre de personne qu'un individu malade peut contaminer. Cela s'explique par le fait que plus le nombre de vaccinés à la première et à la seconde dose augmente, couplé à la levée de restrictions, plus les gens se contaminent mais sans qu'il y ait des formes graves nécessitant l'hospitalisation.

## PREPROCESSING MODELISATION ET PREDICTION

### Chargement des estimateurs, transformers et modèles nécéssaires

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor

### Division de la dataset en train_set et Test_set

In [ ]:
X = df.drop("taux_occupation_sae", axis =1)
y = df["taux_occupation_sae"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

### modèle SDGregressor

In [ ]:
model = make_pipeline(RobustScaler(),
                     SGDRegressor(random_state=0))

#### modification des hyperparamètres

In [ ]:
params = {'robustscaler__with_centering': ['True','False'],
          'sgdregressor__eta0': [0.001,0.0001,0.01]}

In [ ]:
grid = GridSearchCV(model, params, cv=4)

In [ ]:
grid.fit(X_train,y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.score(X_test,y_test)*100

### En utilisant l'algorithme du SDGregressor, la valeur du target est prédite avec 89,9% de précision avec un alpha de descente du gradient oprimal égal à 0.01.

## Modèle KNNeighborsregressor

In [ ]:
model = make_pipeline(RobustScaler(),
                     KNeighborsRegressor())

#### modification des hyperparamètres

In [ ]:
params = {'robustscaler__with_centering': ['True','False'],
          'kneighborsregressor__n_neighbors': [1,2,3,4,5]}

In [ ]:
grid = GridSearchCV(model, params, cv=4)

In [ ]:
grid.fit(X_train,y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.score(X_test,y_test)*100

### En utilisant l'algorithme du KNNeighborsregressor, la valeur du target est prédite avec 98,8% de précision avec un nombre de voisin proche optimal égal à alpha de 1.

## CONCLUSION

### Au terme de cette analyse, il ressort clairement que plus le nombre de personnes vaccinées à la 1ère dose, et encore plus, à la 2ème dose augmente, plus le taux d'occupation des lits dans les hopitaux baisse, ce malgré une hausse de taux de reproductivité du virus, c'est à dire le nombre de personne que contamine une personne infectée, ceci s'explique par le fait que les personnes déjà vaccinées peuvent être contaminées mais sans faire des formes graves de maladie, nécéssitant une hospitalisation.

### L'algorithme de prédiction KNNeighborsregressor, donne un résultat plus satisfaisant que le SGDregressor.